In [22]:
import pandas as pd 
import numpy as np 
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.figure_factory as FF
from wordcloud import WordCloud
import plotly.express as px

In [23]:
itinerarios_bases= pd.read_csv('../Data/Itinerarios/itinerarios_bases.csv')
itinerarios_bases.head()

<ipython-input-23-6017489434ab>:1: DtypeWarning:

Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.



,Unnamed: 0,idplug_base,user_type,idunplug_base,travel_time,idunplug_station,ageRange,idplug_station,unplug_hourTime,zip_code,...,Barrio_Salida,Número de Plazas_Salida,Latitud_Salida,Longitud_Salida,Número_Llegada,Distrito_Llegada,Barrio_Llegada,Número de Plazas_Llegada,Latitud_Llegada,Longitud_Llegada
0,0,5,1,20,154,44,5,40,2019-07-01 00:00:00+00:00,28007,...,01-02 EMBAJADORES,21.0,40.409816,-3.703914,40,01 CENTRO,01-02 EMBAJADORES,24.0,40.413787,-3.704746
1,1,8,1,17,184,57,4,129,2019-07-01 00:00:00+00:00,28045,...,01-05 UNIVERSIDAD,24.0,40.429636,-3.712818,129,07 CHAMBERÍ,07-05 RÍOS ROSAS,24.0,40.438468,-3.698282
2,2,15,1,16,282,53,4,43,2019-07-01 00:00:00+00:00,NaN,...,01-02 EMBAJADORES,24.0,40.408280,-3.700730,43,01 CENTRO,01-02 EMBAJADORES,24.0,40.410089,-3.702545
3,3,18,1,18,388,56,0,13,2019-07-01 00:00:00+00:00,NaN,...,01-06 SOL,24.0,40.418484,-3.703288,13,01 CENTRO,01-05 UNIVERSIDAD,24.0,40.427365,-3.710473
4,4,1,1,20,254,105,0,75,2019-07-01 00:00:00+00:00,NaN,...,04-01 RECOLETOS,24.0,40.427068,-3.687474,75,03 RETIRO,03-05 JERÓNIMOS,24.0,40.408339,-3.678415


In [24]:
cols_rutas=['Origen_destino','Latitud_Salida','Longitud_Salida','Distrito_Salida','Latitud_Llegada','Longitud_Llegada','Distrito_Llegada', 'idplug_station', 'idunplug_station' ]
df_rutas=itinerarios_bases[itinerarios_bases['idplug_station']!=itinerarios_bases['idunplug_station'] ].groupby(cols_rutas)['idplug_base'].count().to_frame().reset_index().sort_values('idplug_base', ascending=False)
df_rutas.rename(columns={'idplug_base': 'viajes'}, inplace=True)

topRutas=df_rutas.head(16)
topRutas['ruta']=topRutas.apply(lambda x: sorted([x.Longitud_Salida, x.Latitud_Salida, x.Longitud_Llegada, x.Latitud_Llegada]), axis=1)
topRutas['ruta']=topRutas['ruta'].apply(lambda x: ' '.join([str(word) for word in x]))
cols_rutas=['Latitud_Salida','Longitud_Salida','Latitud_Llegada','Longitud_Llegada', 'ruta']
def rutas(df): 
    
    
    long_llegada=df['Longitud_Llegada'].iloc[0]
    lat_llegada=df['Latitud_Llegada'].iloc[0]
    long_salida=df['Longitud_Salida'].iloc[0]
    lat_salida=df['Latitud_Salida'].iloc[0]
    num_viajes=df['viajes'].sum()
    cols= ['Latitud_Salida','Longitud_Salida','Latitud_Llegada','Longitud_Llegada', 'viajes']
    datos=[lat_salida, long_salida,lat_llegada, long_llegada, num_viajes ]
    return pd.Series(dict(zip(cols,datos)))
             
topRutasUnicas= topRutas.groupby('ruta').apply(rutas).reset_index()
topRutasUnicas

,ruta,Latitud_Salida,Longitud_Salida,Latitud_Llegada,Longitud_Llegada,viajes
0,-3.6983033 -3.6928901 40.3962687 40.4414845,40.441485,-3.692890,40.396269,-3.698303,1277.0
1,-3.6983033 -3.6955803 40.3962687 40.4132831,40.396269,-3.698303,40.413283,-3.695580,1528.0
2,-3.6992427 -3.6928901 40.4060497 40.4414845,40.441485,-3.692890,40.406050,-3.699243,1270.0
3,-3.7017071 -3.6878512 40.4417195 40.4498448,40.441719,-3.701707,40.449845,-3.687851,1424.0
4,-3.7017071 -3.6893305 40.4417195 40.459282,40.459282,-3.689331,40.441719,-3.701707,613.0
5,-3.7025446 -3.6983033 40.3962687 40.4100889,40.396269,-3.698303,40.410089,-3.702545,1385.0
6,-3.7035911 -3.6983033 40.3962687 40.4463889,40.446389,-3.703591,40.396269,-3.698303,769.0
7,-3.7094093 -3.6878512 40.4156134 40.4498448,40.415613,-3.709409,40.449845,-3.687851,1940.0
8,-3.7094093 -3.6893305 40.4156134 40.459282,40.415613,-3.709409,40.459282,-3.689331,615.0
9,-3.7128181 -3.7114443 40.4112961 40.4296363,40.429636,-3.712818,40.411296,-3.711444,656.0


In [25]:
cols_llegada_estacion=['Longitud_Llegada','Latitud_Llegada', 'Distrito_Llegada' ]
cols_salida_estacion=['Longitud_Salida', 'Latitud_Salida','Distrito_Salida']
cols_estaciones=['Longitud','Latitud', 'Distrito']

estaciones_llegada=topRutas[cols_llegada_estacion]
estaciones_salida=topRutas[cols_salida_estacion]
estaciones_llegada.rename(columns=dict(zip(cols_llegada_estacion,cols_estaciones)) , inplace=True)
estaciones_salida.rename(columns=dict(zip(cols_salida_estacion,cols_estaciones)) , inplace=True)

top10estaciones0=pd.concat([estaciones_salida, estaciones_llegada], axis=0, ignore_index=True )
top10estaciones0['count']=1
def top_estaciones(df): 
    distrito= df['Distrito'].iloc[0]
    count=df['count'].sum()
    cols=['Distrito', 'count']
    datos=[distrito, count]
    return(pd.Series(dict(zip(cols,datos))))
topEstaciones=top10estaciones0.groupby(['Longitud','Latitud']).apply(top_estaciones).reset_index()

topEstaciones

,Longitud,Latitud,Distrito,count
0,-3.712818,40.429636,01 CENTRO,1
1,-3.711444,40.411296,01 CENTRO,1
2,-3.709409,40.415613,01 CENTRO,3
3,-3.703591,40.446389,07 CHAMBERÍ,1
4,-3.702545,40.410089,01 CENTRO,2
5,-3.701707,40.441719,07 CHAMBERÍ,3
6,-3.699243,40.406050,01 CENTRO,2
7,-3.698303,40.396269,02 ARGANZUELA,7
8,-3.695580,40.413283,01 CENTRO,2
9,-3.692890,40.441485,07 CHAMBERÍ,4


In [26]:
fig = px.scatter_mapbox(top10estaciones, lat="Latitud", lon="Longitud", size='count',color='Distrito', width = 1000, height = 900, zoom = 50)
# fig.add_trace(go.Scattermapbox(
#         lat = top10estaciones["Latitud"],
#         lon=top10estaciones["Longitud"],
#         mode='markers',
#         showlegend=False,
#         marker=go.scattermapbox.Marker(
#             size=top10estaciones["count"],
#             color='blue', 
#             opacity=0.7,
#             #size=50,
#              )
#     ))
for i in range(topRutasUnicas.shape[0]): 
    valores=topRutasUnicas.iloc[i,:]

    fig.add_trace(
        go.Scattermapbox(
        mode = "markers+lines",
        lon = [valores['Longitud_Salida'], valores['Longitud_Llegada']],
        lat = [valores['Latitud_Salida'], valores['Latitud_Llegada']],
        line= {'width':valores['viajes']*0.005} ,
        name= 'Ruta '+str(1+i)

        ))
hover_header = 'Estacion: %{text} '

fig.update_layout(
    title='Rutas más concurridas',
    autosize=True,
    hovermode='closest',
    showlegend=True,
    width = 1000,
    height = 1000,
    mapbox=dict(
        bearing=0,
        center=dict(
            lat=40.435,
            lon=-3.69
        ),
        zoom=12.5,
        style= 'carto-positron' # 'open-street-map'
        
    )
)
fig.show()

#### Rutas Mapa

In [29]:
rutas_175=df_rutas[df_rutas['idplug_station']== 175]
rutas_175_top=rutas_175.head(10)

In [31]:
rutas_175_top.rename(columns={'_id': 'viajes'}, inplace=True)

In [32]:
rutas_175_top

,Origen_destino,Latitud_Salida,Longitud_Salida,Distrito_Salida,Latitud_Llegada,Longitud_Llegada,Distrito_Llegada,idplug_station,idunplug_station,viajes
7663,132-175,40.446389,-3.703591,07 CHAMBERÍ,40.396269,-3.698303,02 ARGANZUELA,175,132,769
27301,27-175,40.413283,-3.695580,01 CENTRO,40.396269,-3.698303,02 ARGANZUELA,175,27,677
8277,135-175,40.441485,-3.692890,07 CHAMBERÍ,40.396269,-3.698303,02 ARGANZUELA,175,135,661
30732,43-175,40.410089,-3.702545,01 CENTRO,40.396269,-3.698303,02 ARGANZUELA,175,43,631
29516,38-175,40.411296,-3.711444,01 CENTRO,40.396269,-3.698303,02 ARGANZUELA,175,38,605
83,1-175,40.417111,-3.701998,01 CENTRO,40.396269,-3.698303,02 ARGANZUELA,175,1,554
27095,26-175,40.423831,-3.694542,01 CENTRO,40.396269,-3.698303,02 ARGANZUELA,175,26,496
32798,52-175,40.414518,-3.700784,01 CENTRO,40.396269,-3.698303,02 ARGANZUELA,175,52,384
33629,56-175,40.418484,-3.703288,01 CENTRO,40.396269,-3.698303,02 ARGANZUELA,175,56,381
31148,45-175,40.406949,-3.711138,01 CENTRO,40.396269,-3.698303,02 ARGANZUELA,175,45,375


In [33]:
cols_llegada_estacion=['Longitud_Llegada','Latitud_Llegada', 'Distrito_Llegada' ]
cols_salida_estacion=['Longitud_Salida', 'Latitud_Salida','Distrito_Salida']
cols_estaciones=['Longitud','Latitud', 'Distrito']

estaciones_llegada=rutas_175_top[cols_llegada_estacion]
estaciones_salida=rutas_175_top[cols_salida_estacion]
estaciones_llegada.rename(columns=dict(zip(cols_llegada_estacion,cols_estaciones)) , inplace=True)
estaciones_salida.rename(columns=dict(zip(cols_salida_estacion,cols_estaciones)) , inplace=True)

top175estaciones0=pd.concat([estaciones_salida, estaciones_llegada], axis=0, ignore_index=True )
top175estaciones0['count']=1
def top_estaciones(df): 
    distrito= df['Distrito'].iloc[0]
    count=df['count'].sum()
    cols=['Distrito', 'count']
    datos=[distrito, count]
    return(pd.Series(dict(zip(cols,datos))))
top175estaciones=top175estaciones0.groupby(['Longitud','Latitud']).apply(top_estaciones).reset_index()

top175estaciones

,Longitud,Latitud,Distrito,count
0,-3.711444,40.411296,01 CENTRO,1
1,-3.711138,40.406949,01 CENTRO,1
2,-3.703591,40.446389,07 CHAMBERÍ,1
3,-3.703288,40.418484,01 CENTRO,1
4,-3.702545,40.410089,01 CENTRO,1
5,-3.701998,40.417111,01 CENTRO,1
6,-3.700784,40.414518,01 CENTRO,1
7,-3.698303,40.396269,02 ARGANZUELA,10
8,-3.695580,40.413283,01 CENTRO,1
9,-3.694542,40.423831,01 CENTRO,1


In [35]:
fig = px.scatter_mapbox(top175estaciones, lat="Latitud", lon="Longitud", size='count',color='Distrito', width = 1000, height = 900, zoom = 50)
# fig.add_trace(go.Scattermapbox(
#         lat = top10estaciones["Latitud"],
#         lon=top10estaciones["Longitud"],
#         mode='markers',
#         showlegend=False,
#         marker=go.scattermapbox.Marker(
#             size=top10estaciones["count"],
#             color='blue', 
#             opacity=0.7,
#             #size=50,
#              )
#     ))
for i in range(rutas_175_top.shape[0]): 
    valores=rutas_175_top.iloc[i,:]

    fig.add_trace(
        go.Scattermapbox(
        mode = "markers+lines",
        lon = [valores['Longitud_Salida'], valores['Longitud_Llegada']],
        lat = [valores['Latitud_Salida'], valores['Latitud_Llegada']],
        line= {'width':valores['viajes']*0.005} ,
        name= 'Ruta '+str(1+i)

        ))
hover_header = 'Estacion: %{text} '

fig.update_layout(
    title='Rutas más concurridas',
    autosize=True,
    hovermode='closest',
    showlegend=True,
    width = 1000,
    height = 1000,
    mapbox=dict(
        bearing=0,
        center=dict(
            lat=40.435,
            lon=-3.69
        ),
        zoom=12.5,
        style= 'carto-positron' # 'open-street-map'
        
    )
)
fig.show()

In [3]:
itinerarios_bases['idplug_station']=itinerarios_bases['idplug_station'].astype('str')
itinerarios_bases['idunplug_station']=itinerarios_bases['idunplug_station'].astype('str')

itinerarios_bases['Origen_destino']=itinerarios_bases.apply(lambda x: x.idunplug_station+'-'+x.idplug_station, axis=1)